In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_csv('clustered_accepted_paper_podium.csv', index_col=0)

In [ ]:
df['txt'] = ('TITLE: ' + df.submission_TITLE + '\nABSTRACT: '+df.submission_ABSTRACT).tolist()

In [ ]:
import requests
import json
import time

access_token = "sk-xxx" # please replace with your openai token
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {access_token}'
}

prompt = """
The user will paste the titles and abstracts of 6 papers for a conference session.
Please generate a name for that session.
The session name should be concise, catchy, inspiring and reflecting the content of the 6 papers in that session. 
The session name should be less than 10 words.
Please only return the session name in your response.
Below are a few examples of session names, you do not have to select from these examples:

[
    "Making Sense of COVID Data",
    "New and Intriguing Prediction Models",
    "Enterprise Data Warehouses for Research (EDW4R)",
    "Applied Decision Support for the Greater Good",
    "New Gemstones from Data Mining",
    "Fast Lane to Discovery for Clinical Trial Recruitment",
    "Phenotypes, Models and Cohorts: The Whopper",
    "The Many Faces of Digital Medicine",
    "Building our House: Infrastructure and Informatics",
    "Analyzing Drug Effects and Finding New Uses",
    "COVID Predictions: New Methods and Applications",
    "Patient Engagement in Research and Care",
    "At the Summit of Omics and Informatics Research",
    "Heating up Research with Data and FHIR",
    "Making Sense of Data with Visualizations",
    "Natural Language Processing at Scale",
    "Text Processing for Clinical Notes and Beyond",
    "Applied Natural Language Processing",
    "Full Steam Ahead for Clinical Trials and Real World Data",
    "Social Determinants of Health: Addressing Disparities and Bias",
    "Integrating Patient Perspectives and Ancillary Data Sources",
    "Moving Beyond Bias and Disparities: Informatics for a Better Tomorrow",
    "Infrastructure and New Insights for COVID-19",
    "See What We Did Here? Imaging for Research and Beyond",
    "Models and More in Deep Learning",
    "Putting Machine Learning to Work in Research",
    "Machine Learning for Predictions",
    "Machine Learning for New Insights",
    "Language Matters: New Models and Methods"
]
"""

session_names = {}

for cluster, group in tqdm(df.groupby('GPT_cluster')):

    paper_info = ''
    for i in range(6):
        paper_info += "paper #{}: \n{} \n".format(str(i+1), group.txt.iloc[i])


    post = {
          "model": "gpt-4-1106-preview", # "gpt-3.5-turbo-1106",
          "messages": [
            {"role": "system", "content":prompt},
            {"role": "user", "content":paper_info}]}
    
    response = requests.post("https://api.openai.com/v1/chat/completions", 
                             headers=headers, json=post)

    name = response.json()['choices'][0]['message']['content'].strip()

    session_names[cluster] = name

    time.sleep(5.0)


    

In [ ]:
#gpt-4-1106-preview
session_names

In [ ]:
df = df.merge(pd.DataFrame(list(session_names.items()), columns=['GPT_cluster', 'GPT_session_name']))

In [ ]:
df.to_csv('clustered_accepted_paper_podium.csv')